In [ ]:
import pandas as pd
import pickle as pkl
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### Load the dataset

In [ ]:
data = pd.read_csv('../Master.csv')
data = data[data['actiontype'] == 1]

In [ ]:
feature_set = pkl.load(open('./final_feats_targets.pkl','rb'))
imp_feature_names = [k for k, v in feature_set.items() if v == 3]
X = df[imp_feature_names]
y = df['service_type']
X_norm = MinMaxScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size = 0.30, random_state = 42, stratify = y)

In [ ]:
# Training an Extra tree Classifier and obtaining important features
clf = ExtraTreesClassifier(random_state=0, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

report = classification_report(y_test, y_pred, output_dict=True)
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
names = [feature_names[i] for i in indices]

### Forward feature selection for malicious targets

In [ ]:
ranked_features = names[:50]
y_et = df['service_type']
selected_features = []
score_hist = []
last_score = 0
master_dict = {}
checker = 0

while ranked_features:
    print (len(ranked_features))
    print (selected_features)
    temp = ranked_features
    score_dict = {}
    for _ in range (len (temp)):
        et = ExtraTreesClassifier(n_estimators=100)      
        feat = selected_features + [temp[_]]
        X_et = df[feat]
        X_norm_et = MinMaxScaler().fit_transform(X_et)
        X_train, X_test, y_train, y_test = train_test_split(X_norm_et, y_et, test_size=0.30, random_state=42, stratify=y_et)
        et.fit(X_train, y_train)
        y_pred = et.predict(X_test)
        rpt = classification_report(y_test, y_pred,output_dict=True)
        score_dict[temp[_]] = rpt['macro avg']['f1-score']        
        print ("@"*50)    
        
    Keymax = max(score_dict, key=score_dict.get)
    feat = selected_features + [Keymax]
    X_cross = df[feat]
    X_norm_cross = MinMaxScaler().fit_transform(X_cross)
    X_train, X_test, y_train, y_test = train_test_split(X_norm_cross, y_et, test_size=0.30, random_state=42, stratify=y_et)
    clf = ExtraTreesClassifier(random_state=0, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    max_value = report['macro avg']['f1-score']
    
    print (feat)
    print (classification_report(y_test, y_pred))
    if max_value >= last_score:
        selected_features.append(Keymax)
        last_score = max_value
        master_dict[str(feat)] = {
        'accuracy': report['accuracy'],
        'macro_avg_precision': report['macro avg']['precision'],
        'macro_avg_recall': report['macro avg']['recall'],
        'macro_avg_f1-score': report['macro avg']['f1-score'],
        'weighted_avg_precision': report['weighted avg']['precision'],
        'weighted_avg_recall': report['weighted avg']['recall'],
        'weighted_avg_f1-score': report['weighted avg']['f1-score']
    }
        pd.DataFrame(master_dict).transpose().to_csv('./forward_feature_selection_targets_results.csv')
    ranked_features.remove(Keymax)